<a href="https://colab.research.google.com/github/agrigoridou/Tokenization-Zipf-s-Law-N-gram-Models/blob/main/%CE%92_N_gram_Language_Models_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install nltk numpy

# Προετοιμασία των δεδομένων

Πρώτα, θα φορτώσουμε τα δεδομένα και θα προετοιμάσουμε τα αρχεία για την εκπαίδευση και αξιολόγηση:

In [34]:
import nltk
from nltk.corpus import treebank
from collections import Counter

# Φορτώνουμε τα δεδομένα
nltk.download('treebank')

# Φορτώνουμε τα πρώτα 150 αρχεία για εκπαίδευση και τα υπόλοιπα 49 για αξιολόγηση
train_files = treebank.fileids()[:150]
test_files = treebank.fileids()[150:]

# Λήψη των προτάσεων
train_sents = [sent for file in train_files for sent in treebank.sents(file)]
test_sents = [sent for file in test_files for sent in treebank.sents(file)]


[nltk_data] Downloading package treebank to /root/nltk_data...
[nltk_data]   Package treebank is already up-to-date!


# 1: Προετοιμασία των κειμένων και αντικατάσταση λέξεων που εμφανίζονται λιγότερο από 3 φορές

Αρχικά, αντικαθιστούμε τις λέξεις που εμφανίζονται λιγότερο από 3 φορές με το token <UNK> και προσθέτουμε τα ειδικά tokens <BOS> και <EOS>:

In [35]:
def preprocess_sents(sents):
    # Υπολογισμός συχνοτήτων λέξεων
    all_tokens = [token.lower() for sent in sents for token in sent]
    word_freq = Counter(all_tokens)

    # Δημιουργία λεξιλογίου
    vocab = {word for word, freq in word_freq.items() if freq >= 3}

    # Προετοιμασία των προτάσεων
    processed_sents = []
    for sent in sents:
        new_sent = ['<BOS>'] + [word.lower() if word.lower() in vocab else '<UNK>' for word in sent] + ['<EOS>']
        processed_sents.append(new_sent)

    return processed_sents

# Προετοιμασία των δεδομένων
train_sents = preprocess_sents(train_sents)
test_sents = preprocess_sents(test_sents)


# 2: Δημιουργία bigrams και trigrams με add-k smoothing

Bigram με add-k smoothing (k=1):

In [36]:
from nltk import bigrams

def bigram_model(sents, k=1):
    bigram_freq = Counter()
    unigram_freq = Counter()

    for sent in sents:
        unigrams = sent[:-1]  # Αφήνουμε το <EOS> στο τέλος
        bigrams_list = bigrams(sent)

        # Συχνότητες bigrams και unigrams
        bigram_freq.update(bigrams_list)
        unigram_freq.update(unigrams)

    # Υπολογισμός πιθανότητας bigram με smoothing
    bigram_prob = {}
    vocab_size = len(unigram_freq)

    for (w1, w2), count in bigram_freq.items():
        bigram_prob[(w1, w2)] = (count + k) / (unigram_freq[w1] + k * vocab_size)

    return bigram_prob

# Εκπαίδευση μοντέλου bigram με add-k smoothing (k=1)
bigram_model_1 = bigram_model(train_sents, k=1)


Bigram με add-k smoothing (k=0.01):

In [37]:
# Εκπαίδευση μοντέλου bigram με add-k smoothing (k=0.01)
bigram_model_2 = bigram_model(train_sents, k=0.01)


Trigram με add-k smoothing (k=1):

In [38]:
from nltk import trigrams

def trigram_model(sents, k=1):
    trigram_freq = Counter()
    bigram_freq = Counter()

    for sent in sents:
        bigrams_list = bigrams(sent)
        trigrams_list = trigrams(sent)

        # Συχνότητες trigrams και bigrams
        trigram_freq.update(trigrams_list)
        bigram_freq.update(bigrams_list)

    # Υπολογισμός πιθανότητας trigram με smoothing
    trigram_prob = {}
    vocab_size = len(bigram_freq)

    for (w1, w2, w3), count in trigram_freq.items():
        trigram_prob[(w1, w2, w3)] = (count + k) / (bigram_freq[(w1, w2)] + k * vocab_size)

    return trigram_prob

# Εκπαίδευση μοντέλου trigram με add-k smoothing (k=1)
trigram_model_1 = trigram_model(train_sents, k=1)


Trigram με add-k smoothing (k=0.01):

In [39]:
# Εκπαίδευση μοντέλου trigram με add-k smoothing (k=0.01)
trigram_model_2 = trigram_model(train_sents, k=0.01)


#3: Υπολογισμός perplexity

Ο υπολογισμός της perplexity βασίζεται στη συνάρτηση πιθανότητας για κάθε bigram/trigram:

In [40]:
import math

def calculate_perplexity(model, sents, n=2):
    log_prob = 0
    total_ngrams = 0

    for sent in sents:
        if n == 2:
            ngrams_list = bigrams(sent)
        elif n == 3:
            ngrams_list = trigrams(sent)

        for ngram in ngrams_list:
            if n == 2:
                prob = model.get(ngram, 1e-6)  # Για bigram
            elif n == 3:
                prob = model.get(ngram, 1e-6)  # Για trigram

            log_prob += math.log(prob)
            total_ngrams += 1

    perplexity = math.exp(-log_prob / total_ngrams)
    return perplexity

# Υπολογισμός perplexity για τα μοντέλα
perplexity_bigram_1 = calculate_perplexity(bigram_model_1, test_sents, n=2)
perplexity_bigram_2 = calculate_perplexity(bigram_model_2, test_sents, n=2)
perplexity_trigram_1 = calculate_perplexity(trigram_model_1, test_sents, n=3)
perplexity_trigram_2 = calculate_perplexity(trigram_model_2, test_sents, n=3)

print(f"Perplexity Bigram k=1: {perplexity_bigram_1}")
print(f"Perplexity Bigram k=0.01: {perplexity_bigram_2}")
print(f"Perplexity Trigram k=1: {perplexity_trigram_1}")
print(f"Perplexity Trigram k=0.01: {perplexity_trigram_2}")


Perplexity Bigram k=1: 1129.5836191504993
Perplexity Bigram k=0.01: 338.7903167647698
Perplexity Trigram k=1: 78005.67446892183
Perplexity Trigram k=0.01: 13002.912324562714


# 4: Μετατροπή σε πεζά γράμματα και υποκατάσταση ψηφίων με το ‘#’

In [41]:
def preprocess_lowercase(sents):
    return [[token.lower() for token in sent] for sent in sents]

# Υπολογισμός perplexity για τα μοντέλα με πεζά γράμματα
train_lower = preprocess_lowercase(train_sents)
test_lower = preprocess_lowercase(test_sents)

perplexity_bigram_lower_1 = calculate_perplexity(bigram_model_1, test_lower, n=2)
perplexity_bigram_lower_2 = calculate_perplexity(bigram_model_2, test_lower, n=2)
perplexity_trigram_lower_1 = calculate_perplexity(trigram_model_1, test_lower, n=3)
perplexity_trigram_lower_2 = calculate_perplexity(trigram_model_2, test_lower, n=3)

print(f"Perplexity Bigram (Lowercase) k=1: {perplexity_bigram_lower_1}")
print(f"Perplexity Bigram (Lowercase) k=0.01: {perplexity_bigram_lower_2}")
print(f"Perplexity Trigram (Lowercase) k=1: {perplexity_trigram_lower_1}")
print(f"Perplexity Trigram (Lowercase) k=0.01: {perplexity_trigram_lower_2}")


Perplexity Bigram (Lowercase) k=1: 37681.25351690409
Perplexity Bigram (Lowercase) k=0.01: 15850.960690456799
Perplexity Trigram (Lowercase) k=1: 463091.11400842905
Perplexity Trigram (Lowercase) k=0.01: 248095.06938838182
